In [46]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display
import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize

pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 999


def inf(column):
    return np.log(17608989/float(column))
    
def execute_query(sparqlQuery):
    sparql = SPARQLWrapper("http://zen.cs.rpi.edu:8890/sparql")
    sparql.setQuery(sparqlQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = json_normalize(results["results"]["bindings"])
    results_df = results_df[results_df['o.type'] == "uri"]
    results_df = results_df.drop(['o.datatype','o.type', 'o.xml:lang','p.type'],axis=1)
    results_df = results_df[['p.value','o.value','countS.value']]
    results_df['countS.value'] = results_df['countS.value'].astype(int)
    results_df = results_df[results_df['countS.value'] > 1]
    return results_df


def execute_count_query(obj):
    sparqlQuery = "Select (count(*) as ?countRow)  WHERE { ?s " + "?p  <" + obj + "> }"
    sparql = SPARQLWrapper("http://zen.cs.rpi.edu:8890/sparql")
    sparql.setQuery(sparqlQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = json_normalize(results["results"]["bindings"])
    val = float(results_df['countRow.value'])
    return val


query = """
PREFIX  schema: <http://schema.org/>
PREFIX  res:  <http://dbpedia.org/resource/>
PREFIX  umbel-rc: <http://umbel.org/umbel/rc/>
PREFIX  dbc:  <http://dbpedia.org/resource/Category:>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  yago: <http://dbpedia.org/class/yago/>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  dcterms: <http://purl.org/dc/terms/>
PREFIX  dbpedia-wikidata: <http://wikidata.dbpedia.org/resource/>
PREFIX  dul:  <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>
PREFIX  dc:   <http://purl.org/dc/elements/1.1/>
PREFIX prov: <http://www.w3.org/ns/prov#>

SELECT  ?p ?o (COUNT(DISTINCT ?s) AS ?countS)
WHERE
  { res:Martin_Scorsese
              ?p  ?o .
    ?s        ?p  ?o
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageWikiLink)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageID)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageExternalLink)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageRevisionID)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageLength)) )
    FILTER ( ! strstarts(str(?p), str(owl:sameAs)) )
    FILTER ( ! strstarts(str(?p), str(dbo:viafId)) )
    FILTER ( ! strstarts(str(?p), str(dbo:wikiPageOutDegree)) )
    FILTER ( ! strstarts(str(?p), str(schema:comment)) )
    FILTER ( ! strstarts(str(?p), str(dbo:abstract)) )
    FILTER ( ! strstarts(str(?p), str(rdfs:comment)) )
    FILTER ( ! strstarts(str(?p), str(dbo:alias)) )
    FILTER ( ! strstarts(str(?p), str(rdfs:label)) )
    FILTER ( ! strstarts(str(?p), str(dbo:thumbnail)) )
    FILTER ( ! strstarts(str(?p), str(foaf:name)) )
    FILTER ( ! strstarts(str(?p), str(foaf:surname)) )
    FILTER ( ! strstarts(str(?p), str(dcterms:subject)) )
    FILTER ( ! strstarts(str(?p), str(foaf:depiction)) )
    FILTER ( ! strstarts(str(?p), str(foaf:isPrimaryTopicOf)) )
    FILTER ( ! strstarts(str(?p), str(dbp:hasPhotoCollection)) )
    FILTER ( ! strstarts(str(?p), str(dbp:wordnet_type)) )
    FILTER ( ! strstarts(str(?p), str(prov:wasDerivedFrom)) )
    FILTER ( ! strstarts(str(?p), str(dbo:)) )
    
    
  }
GROUP BY ?p ?o
ORDER BY ASC(?countS)
"""

countQuery = """


"""
results_df = execute_query(query)
results_df.reset_index(inplace=True)
# results_df = results_df.sort_values(by=['countS.value'],ascending=True)

results_df['INF'] = results_df['countS.value'].apply(inf)
results_df['Pop'] = results_df['o.value'].apply(execute_count_query)
# results_df['Rank'] = results_df['Pop'] * results_df['INF']
# results_df.sort_values(by=['Rank'],ascending=False,inplace=True)
# results_df.sort_values(by=['Rank'],ascending=False,inplace=True)
display(results_df)
# results
# for result in results["results"]["bindings"]:
#     print(result["p"]["value"] + " : " + result["o"]["value"]) 
    
# review_df = pd.DataFrame(sparql,index=[0])
# review_df.head()

# from gensim.models import Word2Vec
# model = Word2Vec.load("/home/amar/Data/en_1000_no_stem/en.model")
# model.similarity('woman', 'man')


,index,p.value,o.value,countS.value,INF,Pop
0,11,http://dbpedia.org/property/education,http://dbpedia.org/resource/Cardinal_Hayes_High_School,2,15.990773,68.0
1,16,http://dbpedia.org/property/almaMater,http://dbpedia.org/resource/Tisch_School_of_the_Arts,25,13.465044,918.0
2,18,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/AmericanFilmDirectorsOfItalianDescent,35,13.128572,35.0
3,19,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/FilmTheorists,138,11.756666,138.0
4,20,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/KennedyCenterHonorees,168,11.559956,168.0
5,21,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/AmericanPeopleOfSicilianDescent,204,11.365800,204.0
6,22,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/AmericanMusicVideoDirectors,297,10.990188,297.0
7,23,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/AmericanFilmEditors,351,10.823134,351.0
8,24,http://dbpedia.org/property/almaMater,http://dbpedia.org/resource/New_York_University,358,10.803387,8937.0
9,25,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/AmericanDocumentaryFilmmakers,415,10.655642,415.0
